In [ ]:
import pandas as pd 
import nltk 
import re
from zipfile import ZipFile 


In [ ]:
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> all


       | 
       | Downloading package abc to /root/nltk_data...
       |   Unzipping corpora/abc.zip.
       | Downloading package alpino to /root/nltk_data...
       |   Unzipping corpora/alpino.zip.
       | Downloading package averaged_perceptron_tagger to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger.zip.
       | Downloading package averaged_perceptron_tagger_ru to
       |     /root/nltk_data...
       |   Unzipping taggers/averaged_perceptron_tagger_ru.zip.
       | Downloading package basque_grammars to /root/nltk_data...
       |   Unzipping grammars/basque_grammars.zip.
       | Downloading package biocreative_ppi to /root/nltk_data...
       |   Unzipping corpora/biocreative_ppi.zip.
       | Downloading package bllip_wsj_no_aux to /root/nltk_data...
       |   Unzipping models/bllip_wsj_no_aux.zip.
       | Downloading package book_grammars to /root/nltk_data...
       |   Unzipping grammars/book_grammars.zip.
       | Downloading 


---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

In [ ]:
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip


--2022-09-14 05:54:43--  https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 203415 (199K) [application/x-httpd-php]
Saving to: ‘smsspamcollection.zip’

smsspamcollection.z 100%[===================>] 198.65K   333KB/s    in 0.6s    

2022-09-14 05:54:44 (333 KB/s) - ‘smsspamcollection.zip’ saved [203415/203415]



In [ ]:
with ZipFile("/content/smsspamcollection.zip",'r') as zipobj:
  zipobj.extractall()

In [ ]:
df = pd.read_csv('/content/SMSSpamCollection',sep='\t',names = ['label','message'])

In [ ]:
print(df)

     label                                            message
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...
...    ...                                                ...
5567  spam  This is the 2nd time we have tried 2 contact u...
5568   ham               Will ü b going to esplanade fr home?
5569   ham  Pity, * was in mood for that. So...any other s...
5570   ham  The guy did some bitching but I acted like i'd...
5571   ham                         Rofl. Its true to its name

[5572 rows x 2 columns]


In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer


In [ ]:
# cleaning of data  
ps = PorterStemmer()
corpus = []
for i in range(0,len(df)):
  review = re.sub('[a-zA-z]', " ", df['message'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
  review = " ".join(review)
  corpus.append(review)

In [ ]:
# creating a bag of words model 
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000)
x = cv.fit_transform(corpus).toarray()
y = pd.get_dummies(df['label'])
y = y.iloc[:,1].values

In [ ]:
# train test split
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state = 0)

In [ ]:
# training the model
from sklearn.naive_bayes import MultinomialNB
spam_detection_model = MultinomialNB().fit(x_train,y_train)
y_prediction = spam_detection_model.predict(x_test)

In [ ]:
# accuracy 
from sklearn.metrics import confusion_matrix
confusion_m = confusion_matrix(y_test,y_prediction)
print(confusion_m)

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test,y_prediction)
print(accuracy)

[[954   1]
 [ 89  71]]
0.9192825112107623
